In [2]:
import pandas as pd
from pathlib import Path
#from updates_join_annotations import open_df_from_tsv
#import updates_get_average_scores_per_label as scores


In [3]:
# Read in file and set data type for two columns
filepath = './label_level_df_sorted_cleaned.xlsx'
df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':str})
#df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':list})

#filepath = './label_level_df_sorted_cleaned.tsv'
#df = pd.read_csv(filepath, sep="\t", header=0, encoding="utf-8")



In [4]:
# Sort values and reset index to allow looping later
df.sort_values(by=['file_id', 'start_index', 'end_index', 'Label'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Label1,Label2,Label,file_id,start_index,end_index,tokens,annotators,annotator_count
0,12129,2503,48,54,view\,Patient,view\Patient,2503,48,54,"['Verteld', 'zelf', 'dat', 'hij', 'erg', 'prod...",{'katsburg'},1
1,0,2503,96,98,.B152: Stemming,NaN,.B152: Stemming,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
2,10635,2503,96,98,STM 1,NaN,STM 1,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
3,1,2503,97,98,.B152: Stemming,NaN,.B152: Stemming,2503,97,98,"['was', 'emotioneel']",{'opsomer'},1
4,2,2503,98,98,.B152: Stemming,NaN,.B152: Stemming,2503,98,98,['emotioneel'],"{'swartjes', 'avelli', 'vervaart', 'bos', 'mes...",6


In [6]:
import ast
from ast import literal_eval

def perfectEval(anonstring):
    try:
        ev = ast.literal_eval(anonstring)
        return ev
    except ValueError:
        corrected = "\'" + anonstring + "\'"
        ev = ast.literal_eval(corrected)
        return ev

In [7]:
def convertAnnotators(annos):
    currentannos = annos
    if  type(annos)==str:
        currentannos =literal_eval(label_dict[counter]['annotators'])
        print('current was str')
    elif type(annos)==set:
        #<class 'pandas.core.series.Series'>                        
        currentannos = annos
        print('current was set')
    else:
       # currentannos = perfectEval(str(label_dict[counter]['annotators']))
        currentannos = annos
        print('current was not str', )
    return(currentannos)

In [16]:
#file_id = 2345
#label = 'STM0'
label_dict = dict()

# Loop through file_ids (change later)
# for file_id in set(df['file_id']):
for file_id in [2503]:
    # Create filtered df
    df_filtered = df[df['file_id'] == file_id]
    # Loop through labels and create further filtered df
    for label in set(df_filtered['Label']):
                
        print(file_id, label)
        # df_label has the selection of the rows with the same label
        df_label = df_filtered[df_filtered['Label'] == label]
        df_label.reset_index(inplace=True, drop=True)
        
        try:
            # Set start and end index values
            start_index = df_label['start_index'][0]
            end_index = df_label['end_index'][0]
        except:
            continue
            print('first annotation:', start_index, end_index)

        firstannotation = True
        counter = len(label_dict)+1
        print(df_label.head(1)['tokens'])
        print(df_label.head(1)['annotators'])
        label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': df_label.head(1)['tokens'],
                                           'annotators': convertAnnotators(df_label.head(1)['annotators'])}
        current_range = range(start_index, end_index+1)

        if df_label.shape[0] == 1:
            print('There is only one annotation for this label so we are done.')
            # If only one tag of label in this file, we are done
            continue
 

        # There are multiple annotations so we need to check their spans and merge
        # label through rest of df
        for index, row in df_label[1:].iterrows():
            # Note: df which it is looping over contains only data from one file from one label

            start_temp = row['start_index']
            end_temp = row['end_index']
            print('temp', start_temp, end_temp)


            # Since the df_label is sorted on start & end index, only a few cases need to be checked
            # If the temporary start vallue is in the range of the saved start value
            if start_temp in current_range:
                print('included')
                print('counter', counter)
                if end_temp not in current_range: # And the temp end value is not, then update the end value
                    print('updated')
                    end_index = end_temp
                    ### we need to adapt the range
                    current_range = range(start_index, end_index+1)
                # If the temp end value is also in the range then do nothing and move on.
                ### we update the info for this annotation
                if counter in label_dict:
                    print('counter in label_dict and adding annotators', start_index, end_index)
                    label_dict[counter]['end_index'] = end_index
                    
                    print('label_dict[counter][annotators]', label_dict[counter]['annotators'])
                    print(type(label_dict[counter]['annotators']))
                          
                    currentannos = convertAnnotators(label_dict[counter]['annotators'])
                    print('currentannos', currentannos)
                    print('type currentannos', type(currentannos))
                          
                    newannos = literal_eval(row['annotators'])
                    print(type(currentannos))
                    print('current', currentannos)
                    print(type(newannos))
                    print('new', newannos)
                    if type(currentannos)==set:
                        currentannos.update(newannos)
                    else:
                        currentannos = set(currentannos)
                        currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos
                    print('Extended annotators:',label_dict[counter]['annotators'])
                else:
                    print('counter not in label_dict', start_index, end_index)
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
                
            else:
                
                ### We have a new first annotation with start and range
                start_index = start_temp
                end_index = end_temp
                current_range = range(start_index, end_index+1)
                counter = len(label_dict)+1
                label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}

                print('new first annotation:', start_index, end_index)
                print('counter', counter)
                
            # For last entry
            if index == len(df_label)-1:
                print('last, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    
                    currentannos = convertAnnotators(label_dict[counter]['annotators'])
                    print('currentannos', currentannos)
                    print('type currentannos', type(currentannos))
                          
                    newannos = literal_eval(row['annotators'])
                    print(type(currentannos))
                    print('current', currentannos)
                    print(type(newannos))
                    print('new', newannos)
                    
                    currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos

                    print('Extended annotators:',label_dict[counter]['annotators'])
                   
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
    


2503 stm\reaction
0    ['schoonzoon', 'hoort', 'morgen', 'de', 'uitsl...
Name: tokens, dtype: object
0    {'swartjes', 'meskers', 'avelli', 'katsburg'}
Name: annotators, dtype: object
current was not str
temp 102 104
included
counter 1
counter in label_dict and adding annotators 100 108
label_dict[counter][annotators] 0    {'swartjes', 'meskers', 'avelli', 'katsburg'}
Name: annotators, dtype: object
<class 'pandas.core.series.Series'>
current was not str
currentannos 0    {'swartjes', 'meskers', 'avelli', 'katsburg'}
Name: annotators, dtype: object
type currentannos <class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
current 0    {'swartjes', 'meskers', 'avelli', 'katsburg'}
Name: annotators, dtype: object
<class 'set'>
new {'vervaart'}
Extended annotators: {'vervaart', "{'swartjes', 'meskers', 'avelli', 'katsburg'}"}
last, stored
current was set
currentannos {'vervaart', "{'swartjes', 'meskers', 'avelli', 'katsburg'}"}
type currentannos <class 'set'>
<class 'set'>


In [18]:
#### The next function could be differentiated per label
#- gemiddelde aantal gedeelde annotaties per label
#- hoeveel annotaties per student
#- hoe vaak is student outlier
#- confusion matrix
from collections import Counter

labels = []
annotators = []
annotation_total = 0
for key, value in label_dict.items():
    label = label_dict[key]['label']
    labels.append(label)
    key_annotators=label_dict[key]['annotators']
    for anno in key_annotators:
        annotators.append(anno)
    annotation_total += len(key_annotators)

print('Annotations per label', Counter(labels))
print('Annotations per annotator',Counter(annotators))
print('Total annotations:', len(label_dict.items()))
print('Total student annotations:', annotation_total)
print('Average agreement of annotations', annotation_total/(len(label_dict.items())))

Annotations per label Counter({'STM 1': 2, '.B152: Stemming': 2, 'view\\Patient': 2, 'stm\\reaction': 1, 'STM 0': 1})
Annotations per annotator Counter({'vervaart': 5, 'katsburg': 5, 'avelli': 4, 'meskers': 4, 'bos': 4, 'swartjes': 4, "{'vanderpas'}": 3, 'opsomer': 2, "{'swartjes', 'meskers', 'avelli', 'katsburg'}": 1, 'vanderpas': 1, "{'katsburg'}": 1})
Total annotations: 8
Total student annotations: 34
Average agreement of annotations 4.25


In [59]:
df_soft = pd.DataFrame.from_dict(label_dict, orient='index')

In [60]:
df_soft.head()

,file_id,start_index,end_index,label,tokens,annotators
1,2503,111,116,STM 0,"0 ['maakt', 'zich', 'hier', 'veel', 'zorgen...","{meskers, {'vanderpas'}, avelli}"
2,2503,100,108,stm\reaction,"0 ['schoonzoon', 'hoort', 'morgen', 'de', '...","{vervaart, {'swartjes', 'meskers', 'avelli', '..."
3,2503,96,98,STM 1,"0 ['Mw', 'was', 'emotioneel'] Name: tokens,...","{avelli, bos, meskers, {'vanderpas'}, opsomer,..."
4,2503,111,117,STM 1,"['maakt', 'zich', 'hier', 'veel', 'zorgen', 'om']","{{'vanderpas'}, vervaart}"
5,2503,96,98,.B152: Stemming,"0 ['Mw', 'was', 'emotioneel'] Name: tokens,...","{meskers, {'vanderpas'}, avelli, swartjes, kat..."


In [110]:
df_soft.to_excel('softboundary_output_file.xlsx')

In [ ]:
# file_id = 1234
# label = 'Stemming'
# label_dict = {label: dict()}
# # Set impossible start and end index values
# start_index = -2
# end_index = -1

# for index, row in df_label.iterrows():
#     # Note: df which it is looping over contains only data from one file from one label
#     #counter = len(label_dict[label])+1
    
#     start_temp = row['start_index']
#     end_temp = row['end_index']
#     print('temp', start_temp, end_temp)
    
#     current_range = range(start_index, end_index+1)
    
#     # If temp start and end are in currently stored start-end range, change nothing
# #     if start_temp in current_range and end_temp in current_range:
# #         continue
#     # If temp start in the current range (there is overlap), and end temp is not in the current range, update the end index
#     if start_temp in current_range and not end_temp in current_range:
#         print('a')
#         end_index = end_temp
#     # Elif temp end in the current range (there is overlap), and start temp is not in the current range, update the start index
#     elif end_temp in current_range and not start_temp in current_range:
#         print('b')
#         start_index = start_temp
#     # Elif if the temp range encompasses the stored/current range, update both start and end indices
#     elif start_index in range(start_temp, end_temp+1) and end_index in range(start_temp, end_temp+1):
#         print('c')
#         start_index = start_temp
#         end_index = end_temp
        
#     elif all([not start_temp in current_range,
#               not end_temp in current_range,
#               start_index >= 0,  # As to not save initial values
#               end_index >=0]): 
#         # If new start and end indices are disjoint from stored indices, then save and restart
#         print('d')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#         start_index = start_temp
#         end_index = end_temp
        
#     if index == len(df_label)-1:
#         print('yes')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#     print('stored', start_index, end_index)